Click the button below if you want to see the code behind the widgets. If you do click  and want to go back to the 'no-code' view, scroll down until you find the 'hide code' button.

In [1]:
"""
MIT License

Copyright (c) 2020 Sylvain Barde - University of Kent

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
"""

import ipywidgets as widgets
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from matplotlib import cm
from matplotlib.colors import ListedColormap


import numpy as np
from matplotlib import rcParams
from IPython.display import display, HTML

javascript_functions = {False: "hide()", True: "show()"}
button_descriptions  = {False: "Click to show code", True: "Click to hide code"}

def toggle_code(state):
    """
    Toggles the JavaScript show()/hide() function on the div.input element.
    """

    output_string = "<script>$(\"div.input\").{}</script>"
    output_args   = (javascript_functions[state],)
    output        = output_string.format(*output_args)

    display(HTML(output))
    
def button_action(value):
    """
    Calls the toggle_code function and updates the button description.
    """
    state = value.new

    toggle_code(state)

    value.owner.description = button_descriptions[state]


def utility_widget(curve_init = 'CD', a_init = 0.5, xMin_init = 0, xMax_init = 10, xVal_init = 5, yVal_init = 5, 
                      elev_init = 30, azim_init = 290, zCut_init = False):
    
    # Declare widgets for interactive input
    curve_list = widgets.Dropdown(options=['CD'],
                                value = curve_init,
                                description='function:',
                                disabled=False)
    a_slider = widgets.FloatSlider(min=0,
                                 max=1,
                                 step=0.1,
                                 description=r'$\alpha$:',
                                 value = a_init,
                                 continuous_update =False)
    xMin_slider = widgets.IntSlider(min=-100,
                                 max=0,
                                 step=1,
                                 description=r'Min. $x$,$y$:',
                                 value = xMin_init,
                                 continuous_update =False)    
    xMax_slider = widgets.IntSlider(min=0,
                                 max=100,
                                 step=1,
                                 description=r'Max. $x$,$y$:',
                                 value = xMax_init,
                                 continuous_update =False)
    xVal_slider = widgets.FloatSlider(min=-100,
                                 max=100,
                                 step=1,
                                 description=r'$x$ coord:',
                                 value = xVal_init,
                                 continuous_update =False)
    yVal_slider = widgets.FloatSlider(min=-100,
                                 max=100,
                                 step=1,
                                 description=r'$y$ coord:',
                                 value = yVal_init,
                                 continuous_update =False)
    elev_slider = widgets.FloatSlider(min=-90,
                                 max=90,
                                 description= 'Elevation:',
                                 value = elev_init,
                                 continuous_update =False)
    azim_slider = widgets.FloatSlider(min=0,
                                 max=360,
                                 description= 'Azimuth:',
                                 value = azim_init,
                                 continuous_update =False)
    zCut_check = widgets.Checkbox(value = zCut_init,
                                   description='Fix $U$',
                                   disabled=False,
                                   indent=True)    
    
    # Link widgets as required
    widgets.jslink((xMax_slider,'value'),(xVal_slider,'max'))
    widgets.jslink((xMin_slider,'value'),(xVal_slider,'min'))
    widgets.jslink((xMax_slider,'value'),(yVal_slider,'max'))
    widgets.jslink((xMin_slider,'value'),(yVal_slider,'min'))
    
    def utility_plot(curve, a, xMin, xMax, xVal, yVal, elev, azim , zCut):

        fundict = {'CD': lambda x, y: (x**a)*(y**(1-a)),
                  }
        fundictAlt = {'CD': lambda x, z: (x**(-a/(1-a)))*(z**(1/(1-a))),
                  }

        f = fundict[curve]
        fAlt = fundictAlt[curve]

        res = 50
        xFull = np.linspace(1e-3, xMax, res)
        yFull = np.linspace(1e-3, xMax, res)
        XFull, YFull = np.meshgrid(xFull, yFull)
        ZFull = f(XFull, YFull)

        zVal = f(xVal, yVal)
        zMax = max(ZFull.flatten())

        if zCut is True:
            resZ = int(np.floor((zVal/zMax)*res))
            z = np.linspace(0, zVal, resZ)
            zCover = (zVal/zMax)
            viridis = cm.get_cmap('viridis', 512)
            myCmap = ListedColormap(
                       viridis(np.linspace(0, zCover, 256))
                       )
        else:
            z = np.linspace(0, zMax, res)
            myCmap = 'viridis'

        X, Z = np.meshgrid(xFull, z)
        Y = fAlt(X, Z)
                
        Y = np.clip(Y,0,xMax)
        Z = f(X,Y)
        
        # Create figure
        mrkrSize = 2*rcParams['lines.markersize'] ** 2
        fig = plt.figure(figsize=(20,10))

        # Plot 3D utility function
        ax = fig.add_subplot(1, 2, 1, projection='3d')
        ax.plot_surface(X, Y, Z, cmap=myCmap)
        
        if zCut is True:
            xIC = X[-1,:]
            yIC = Y[-1,:]
            zIC = Z[-1,:]
            maskIC = yIC < xMax
            ax.plot3D(xIC[maskIC], yIC[maskIC], zIC[maskIC], 'r', linewidth=2)
            
        ax.plot3D([xVal], [yVal], [f(xVal, yVal)], marker = 'o', color = 'r', zorder = 5)

        ax.set_xlim(xMin,xMax)
        ax.set_ylim(xMin,xMax)
        ax.set_zlim(0,1.5*zMax)
        ax.view_init(elev, azim)
        ax.set_xlabel(r'$x$', fontdict = {'fontsize': 25},position=(1, 0))
        ax.set_ylabel(r'$y$', fontdict = {'fontsize': 25},position=(0, 1))
        ax.set_zlabel(r'$U(x,y)$', fontdict = {'fontsize': 25},position=(0, 1), rotation=0)
        
        # Plot Indifference curve map
        ax = fig.add_subplot(1, 2, 2)
        numIC = 6
        zStepVec = np.linspace(zMax/numIC, zMax-1/numIC, numIC)
        for zStep in zStepVec:
            ax.plot(xFull, fAlt(xFull, zStep), 'k--', alpha=0.3)
        
        if zCut is True:
            zCurr = f(xVal,yVal)
            ax.plot(xFull, fAlt(xFull, zCurr), 'r', linewidth=2, alpha=0.6,
                    label=r'Current indifference curve')

        # Add markers for the price/quantity points, with dotted lines
        ax.scatter(xVal, yVal, s=mrkrSize, c='k', alpha=0.6,
                    label='Selection')
        ax.plot([0,xVal],[yVal,yVal],'r--',linewidth=1)
        ax.plot([xVal,xVal],[0,yVal],'r--',linewidth=1)

        ax.autoscale(enable=True, axis='both', tight=True)
        ax.set_ylim(top = xMax, bottom = 0)
        ax.set_xlim(right = xMax, left = 0)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.set_xlabel(r'$x$', fontdict = {'fontsize': 25},position=(1, 0))
        ax.set_ylabel(r'$y$', fontdict = {'fontsize': 25},position=(0, 1), rotation=0)
        ax.plot(1, 0, ">k", transform=ax.get_yaxis_transform(), clip_on=False)
        ax.plot(0, 1, "^k", transform=ax.get_xaxis_transform(), clip_on=False)
        ax.tick_params(labelsize=20)
        
        plt.tight_layout()

    out = widgets.interactive_output(utility_plot, {'curve': curve_list,
                                                       'a': a_slider,
                                                       'xMin': xMin_slider,
                                                       'xMax': xMax_slider,
                                                       'xVal': xVal_slider,
                                                       'yVal': yVal_slider,
                                                       'elev': elev_slider,
                                                       'azim': azim_slider,
                                                       'zCut': zCut_check})
        
    output = widgets.VBox([out,
                  widgets.HBox([curve_list,
                                a_slider]),
                  widgets.HBox([xMin_slider,
                                xMax_slider,
                                xVal_slider,
                                yVal_slider]),
                  widgets.HBox([elev_slider, 
                                azim_slider,
                                zCut_check])
                          ])
    display(output)

In [2]:
state = False
toggle_code(state)

button = widgets.ToggleButton(state, description = button_descriptions[state])
button.observe(button_action, "value")

display(button)

ToggleButton(value=False, description='Click to show code')

### Utility functions, indifference curves and total derivatives

This notebook will illustrate multivariate functions and total derivatives in the context of the utility function. This is because maximisation of utility subject to a given bugdet is a standard problem that students in economics learn at some point in their training, and we will be looking at this next week.

The notebook therefore lays the ground a bit for the stuff we'll see next week. It also allows us to see a neat application of the total derivative of the utility function, in that it allows us to easily find the slope of the indifference curves, a problem which might otherwise be tricky to solve.


#### Utility as a multivariate function

The utility function of a consumer is the function that tells us how much *utility* (i.e. satisfaction) they get from consuming a 'basket' or various goods: for example, it will tell you how much satisfaction you might get from going out for a Nando's with some friends, followed by a movie with popcorn. At first glance, this seems like a silly concept: we can all agree that you might derive satisfaction from this basket, but how on earth can this be *measured*?

The answer comes from the **marginal** approach to economics we saw in last week's notebooks: we don't need to be able to measure it, we only need to know if it increases or falls. So for our 'night out' example, you might not be able to tell me **how much satisfaction** you get from that Nando's + movie + popcorn bundle, but you'll definitely be able to tell me if it increases or falls if I replace the popcorn with nachos. Clearly, everybody has different preferences, so we all have different utility functions, but we should all be able to rank those alternatives.

The plot on the left below shows a very specific type of multivariate utility function, often referred to as a 'Cobb-Douglas' function after the name of the two economists who came up with it. The nice thing about this function is that preferences are governed by a single parameter. While this may seem very simplistic, it turns out that in a lot of cases this works pretty well, which is why the function is popular. Given the consumption of two quantities of goods $x$ and $y$, the utility is given by: 

$$U(x,y) = x^\alpha y^{1-\alpha}$$

We have a function that accepts 2 inputs ($x$ and $y$) and returns a quantity, utility, hence the 3D plot. Here $\alpha$ is the parameter that governs the relative preference for goods $x$ and $y$. The closer $\alpha$ is to 1, the more you prefer $x$ relative to $y$. The reverse is true the closer $\alpha$ is close to 0. Feel free to modify this parameter in the widget to see its impact on the shape of the utility function.


In [3]:
utility_widget()

#### Indifference curves: a 2D map of a 3D function.

While the 3D plot can be helpful for visualising or understanding how the function works, it is also useful to be able to represent the function in 2 dimensions. After all, you can't easily print in 3 dimensions on the page of a book...

This is where indifference curves come in: If you select a value for $x$ and a value for $y$, the level of utility achieved is displayed with a red point on our utility function. Selecting the 'Fix $U$' will take a slice of the utility function along the vertical axis passing through that point. The red intersection that is revealed shows all the points in the utility function that have the same level of utility as the point itself.

The key idea is the same as for the 3D diagrams shown in the videos on multivariate functions: if you take a slice of a 3D function along any axis, the intersection is a univariate function that can be represented in 2D. We already did this for derivatives, when we fixed $x$ or $y$ to reveal a univariate function we could differentiate more easily. The only difference here is that we're taking a slice along the $z$ axis. This curve, shown in red in the diagram on the right, is called an ***indifference curve***. This is because by construction, all the combinations of $x$ and $y$ on that line provide the same utility (we fixed $U$, remember) therefore you are indifferent between them.

By changing your selection for $x$ and $y$, you can move about and generate many different indifference curves, each one showing a different level of utility. Such a collection is called an **indifference map**, and the result is the same as for regular maps. As you can see below, altitude is recorded on a regular map in the same way, using a set of contour lines to show locations that share the same altitude. 

![Contours on a map](https://getoutside.ordnancesurvey.co.uk/site/uploads/images/2018champs/Blog%20imagery/advanced_contours_guide_streams.jpg)

#### The slope of the indifference curve using the total derivative of the utility function.

So we now have a bunch of curves that can be used to represent a 3D function in two dimensions. They are smooth, and they are non-linear, so they should each have a slope given by a derivative. The question is, how do we calculate these derivative? We don't have an equation for these indifference curves! Furthermore, there are potentially an infinite number of indifference curves, which complicates the problem a bit.

**Why** we need the slope of the utility function is something I will explain at the end. Right now, I want to focus on showing **how** to calculate it. This is where the concept of total derivative comes in very handy. So looking back at your lecture notes, you will remember that the total derivative of the utility function is:

$${\rm d}U(x,y) = \frac{\partial U(x,y)}{\partial x}{\rm d}x \; + \; \frac{\partial U(x,y)}{\partial y}{\rm d}y$$

The first thing we can do is replace the partial derivatives w.r.t. $x$ and $y$ by the economic concept of marginal utility. This is the increase in utility, or satisfaction, that you experience if you consume a tiny bit more of $x$ or $y$. This will simplify the notation a bit by getting rid of the partial derivative notation:

$${\rm d}U(x,y) = mU_x{\rm d}x \; + \; mU_y{\rm d}y$$

The important step is to realise that on a given indifference curve, the total amount of utility you experience is constant. Again, remember we generated the indifference curves by slicing the function along the $z$ axis, in other words by keeping utility constant. This means that on an indifference curve, we have ${\rm d}U(x,y) = 0$ by construction. This tells us that on all indifference curves we have:

$$mU_x{\rm d}x \; + \; mU_y{\rm d}y = 0$$

All we have to do now is rearrange the expression to get the slope of the indifference curve that passes through $any$ point we choose on the diagram:

$$ mU_y{\rm d}y \; = \; - mU_x{\rm d}x$$

$$ \frac{{\rm d}y}{{\rm d}x} \; = \; - \frac{mU_x}{mU_y}$$

What we see is that the slope of an indifference curve at a given point is equal to the ratio of marginal utilities at that point. This makes empirical sense: imagine you are consuming two goods $x$ and $y$, if I reduce the amount of $x$ you have by one unit, you will lose a bit of satisfaction. Suppose I offer to compensate you by giving you more of good $y$ instead. How much do I need to give you? well that depends on how much you value good $y$ relative to good $x$. If you value $y$ a **lot**, then I don't need to give you much more $y$ to offset the loss of $x$. If instead you **don't** like $y$ very much, I'd have to give you a lot more $y$ to make up the loss in $x$.


#### Wrapping up...

This example was designed to illustrate how powerful total derivatives are when used effectively. The key aspect here is that we have found a formula for the derivative of the indifference curve:
- Without picking a specific indifference curve. In fact, the formula works for all indifference curves!
- Without having an equation for an indifference curve.
- Without much work at all. Just two little lines to re-arrange.

Another purpose was to provide a little bit of an introduction to the framework economists use when modeling how agents (consumers in this case) make their choices according to their preferences. We will be looking at this next week, and this is where we will see why know the slope of an indifference curve is important. As we will discover, once you add the fact that consumers have a limited budget and can therefore only pick baskets of goods they can afford, figuring out which is the best basket to pick will depend in great part on the ratio of marginal utilities for that basket, i.e. the slope of the indifference curve. 